In [3]:
#\-- IMPORT MODULES, CLASSES AND METHODS --/#

import zipfile                          #############################
import os                               # || FILE SYSTEM / UTILS || #
import copy                             #############################
from prettytable import PrettyTable

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

import numpy as np                  ###################################
import pandas as pd                 # || EXPLORATIVE DATA ANALYSIS || #
import matplotlib.pyplot as plt     ###################################
# https://towardsdatascience.com/handling-missing-data-like-a-pro-part-3-model-based-multiple-imputation-methods-bdfe85f93087 NumPyro, impyute,

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

import sklearn
import re
import importlib
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from sklearn import naive_bayes                         #########################
from sklearn import neural_network                      #  |-----------------|  #
from sklearn import svm                                 # || MODEL SELECTION || #
from sklearn import tree                                #  |-----------------|  #
from sklearn import linear_model                        #########################

# from PrunedCV import PrunedCV

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

from sklearn.model_selection import StratifiedKFold     ##########################
from sklearn.model_selection import ParameterGrid       # || MODEL VALIDATION || #
                                                        ##########################

%matplotlib inline


In [23]:
#\-- SET ENVIRONMENT --/#
# Before starting we need to store the data properly. We define an ad-hoc folder where we will store everything.
main_PATH = os.getcwd()

# We check whether we already have the data.                        
if 'data' not in os.listdir():                                      
                                                                    
    # Unzip files.
    with zipfile.ZipFile(r'summer_project_dataset.zip') as zip_ref:

        os.mkdir(main_PATH + '/data')   # We create the 'data' directory,
        os.chdir(main_PATH + '/data')   # we change directory,
    
        data_PATH = os.getcwd()         # we get the data path
        zip_ref.extractall(data_PATH)   # and we unzip there.       #####################
                                                                    # || FILE SYSTEM || #    
    file_PATH = data_PATH + '/summer_project_dataset'               #####################

else:

    # We just build the paths.
    data_PATH = main_PATH + '/data'
    file_PATH = data_PATH + '/summer_project_dataset'

# Finally, we go back to our main path.
os.chdir(main_PATH)

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# We also set a seed for reproducibility purposes.      #####################
SEED = 42                                               # || RANDOM SEED || #
np.random.seed(SEED)                                    #####################

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# LaTeX style plots.
plt.rcParams['figure.figsize'] = (20, 8)
# plt.rcParams['text.usetex']    = True         ############################
# plt.rcParams['font.family']    = 'serif'      # || DEFAULT PARAMETERS || #
# plt.rcParams['font.size']      = '10'         ############################

pd.options.display.max_rows = 60
# pd.set_option('display.max_rows', 10)


In [44]:
#\-- DATASET LOADING AND PREPROCESSING --/#
# Here we can optionally specify the names of the features and their type; warmly suggested.
# cols_names = ['']
data_types = {
              'url' : str, 'timedelta' : int, 'shares' : float, 'data_channel' : str, 'weekday' : str, 
              
              'n_tokens_title'          : float, 'n_tokens_content'       : float, 'n_unique_tokens' : float, 'n_non_stop_words' : float,
              'n_non_stop_unique_tokens': float, 'average_token_length' : float,

              'num_hrefs' : float, 'num_self_hrefs' : float, 'num_imgs' : float, 'num_videos' : float,
              
              'kw_min_min' : float, 'kw_max_min'   : float, 'kw_avg_min'   : float, 'kw_min_max' : int, 'kw_max_max'     : float,
              'kw_avg_max' : float, 'kw_min_avg' : float, 'kw_max_avg' : float, 'kw_avg_avg' : float, 'num_keywords' : float,
              
              'self_reference_min_shares' : float, 'self_reference_max_shares' : float, 'self_reference_avg_sharess' : float,
              
              'LDA_00' : float, 'LDA_01' : float, 'LDA_02' : float, 'LDA_03' : float, 'LDA_04' : float,
              
              'global_subjectivity' : float, 'global_sentiment_polarity' : float, 'global_rate_positive_words' : float, 'global_rate_negative_words' : float,
              
              'rate_positive_words' : float, 'rate_negative_words' : float,
              
              'avg_positive_polarity' : float, 'min_positive_polarity' : float, 'max_positive_polarity' : float, 'avg_negative_polarity' : float,
              'min_negative_polarity' : float, 'max_negative_polarity' : float,

              'title_subjectivity' : float, 'title_sentiment_polarity' : float, 'abs_title_subjectivity' : float, 'abs_title_sentiment_polarity' : float,
              }                                                    
                                                                   
                                                                   
data = pd.read_csv(file_PATH + r'/development.csv',                 
                   usecols = lambda column: column != 'id', dtype = data_types)              
# data.info(), data.describe()

# We just check the data types. This is important because it tells us if everything went fine.
pd.options.display.max_rows = 60
# data.dtypes
data
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# We select only the columns that are filled with float64. We do this because in this branch we are doing no preprocessing.
# X = data.loc[:, data.columns != 'review/overall']
# X = X.loc[:, 'review/appearance' : 'review/taste']
# y = data.loc[:, 'review/overall']                                           #######################
#                                                                             # || PREPROCESSING || #
# # We do this just to try StratifiedKFold. We need categorical variables.    #######################
# X = np.array(X)
# new_y = copy.deepcopy(y)
# new_y = np.array(np.where(new_y > 3, 1, 0))  # Here we just encode the numerical label into a categorical one.

# X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size = 0.20, random_state = SEED,
#                                                         shuffle = True, stratify = new_y
#                                                    )



,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,data_channel,weekday
0,http://mashable.com/2014/09/08/safest-cabbies-...,121,12.0,1015.0,0.422018,1.0,0.545031,10.0,6.0,33.0,...,-0.160714,-0.500000,-0.071429,0.000000,0.000,0.500000,0.000,2900.0,bus,tuesday
1,http://mashable.com/2013/07/25/3d-printed-rifle/,532,9.0,503.0,0.569697,1.0,0.737542,9.0,0.0,NaN,...,-0.157500,-0.250000,-0.100000,0.000000,0.000,0.500000,0.000,1300.0,tech,thursday
2,http://mashable.com/2013/10/30/digital-dinosau...,435,9.0,232.0,0.646018,1.0,0.748428,12.0,3.0,4.0,...,-0.427500,-1.000000,-0.187500,0.000000,0.000,0.500000,0.000,17700.0,lifestyle,wednesday
3,http://mashable.com/2014/08/27/homer-simpson-i...,134,12.0,171.0,0.722892,1.0,0.867925,9.0,5.0,0.0,...,-0.216667,-0.250000,-0.166667,0.400000,-0.250,0.100000,0.250,1500.0,bus,wednesday
4,http://mashable.com/2013/01/10/creepy-robotic-...,728,11.0,286.0,0.652632,1.0,0.800000,5.0,2.0,NaN,...,-0.251786,-0.500000,-0.100000,0.200000,-0.100,0.300000,0.100,1400.0,tech,thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31710,http://mashable.com/2014/11/30/star-wars-guard...,37,11.0,440.0,0.564103,1.0,0.718978,10.0,2.0,NaN,...,-0.209167,-0.316667,-0.050000,0.000000,0.000,0.500000,0.000,1000.0,world,tuesday
31711,http://mashable.com/2014/11/14/uk-floods/,52,14.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.666667,-0.700,0.166667,0.700,11000.0,lifestyle,monday
31712,http://mashable.com/2014/09/08/paypal-bitcoin-...,121,9.0,969.0,0.489583,1.0,0.710623,6.0,5.0,2.0,...,-0.400000,-1.000000,-0.050000,0.000000,0.000,0.500000,0.000,2400.0,tech,tuesday
31713,http://mashable.com/2013/08/23/mashable-androi...,503,11.0,1976.0,0.412308,1.0,0.621080,21.0,3.0,1.0,...,-0.323413,-1.000000,-0.050000,0.700000,-0.400,0.200000,0.400,6000.0,lifestyle,friday


In [38]:
data

NameError: name 'ata' is not defined